In [1]:
import polars as pl
from influxdb_client.client.influxdb_client import InfluxDBClient
from influxdb_client import __version__ as influxdb_version
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

# InfluxDB connection
url = "http://192.168.1.17:8086"
token = "_gCwoPkdKM6aF-SXB9crHatd4lKFBXKVcBHhyHMgCqTI2GvsxWkO6qooThciZp49Ra-dQR9B6jOd_ve5isMcHw=="
org = "mush"
bucket = "bucket"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [12]:
from datetime import datetime, timedelta

start_date = datetime(2025, 3, 7)
end_date = datetime(2025, 3, 14)
chunk_size = timedelta(days=1)

all_records = []
current_date = start_date

while current_date < end_date:
    chunk_end = min(current_date + chunk_size, end_date)
    
    print(f"Fetching data from {current_date} to {chunk_end}")
    
    query = f'''
    from(bucket: "bucket")
      |> range(start: {current_date.strftime("%Y-%m-%dT%H:%M:%SZ")}, stop: {chunk_end.strftime("%Y-%m-%dT%H:%M:%SZ")})
      |> filter(fn: (r) => r["_measurement"] == "mqtt_consumer")
      |> filter(fn: (r) => r["topic"] =~ /(temperature|humidity)$/)
    '''
    
    result = query_api.query(query=query)
    
    chunk_records = []
    for table in result:
        for record in table.records:
            chunk_records.append({
                'time': record.get_time(),
                'measurement': record.values.get('measurement'),
                'value': record.get_value(),
                'sensor': record.values.get('device'),
                'topic': record.values.get('topic')
            })
    
    print(f"Got {len(chunk_records)} records for this chunk")
    all_records.extend(chunk_records)
    current_date = chunk_end

print(f"Total records: {len(all_records)}")
df = pl.DataFrame(all_records)

Fetching data from 2025-03-07 00:00:00 to 2025-03-08 00:00:00
Got 59902 records for this chunk
Fetching data from 2025-03-08 00:00:00 to 2025-03-09 00:00:00
Got 59937 records for this chunk
Fetching data from 2025-03-09 00:00:00 to 2025-03-10 00:00:00
Got 59928 records for this chunk
Fetching data from 2025-03-10 00:00:00 to 2025-03-11 00:00:00
Got 59926 records for this chunk
Fetching data from 2025-03-11 00:00:00 to 2025-03-12 00:00:00
Got 59936 records for this chunk
Fetching data from 2025-03-12 00:00:00 to 2025-03-13 00:00:00
Got 59938 records for this chunk
Fetching data from 2025-03-13 00:00:00 to 2025-03-14 00:00:00
Got 59932 records for this chunk
Total records: 419499


In [13]:
df

time,measurement,value,sensor,topic
"datetime[μs, UTC]",str,f64,str,str
2025-03-07 00:00:11.618779 UTC,"""humidity""",94.13,"""bme280_0""","""mush/controllers/C1/sensors/bm…"
2025-03-07 00:00:27.250501 UTC,"""humidity""",94.14,"""bme280_0""","""mush/controllers/C1/sensors/bm…"
2025-03-07 00:00:42.814878 UTC,"""humidity""",94.16,"""bme280_0""","""mush/controllers/C1/sensors/bm…"
2025-03-07 00:00:58.164877 UTC,"""humidity""",94.16,"""bme280_0""","""mush/controllers/C1/sensors/bm…"
2025-03-07 00:01:13.661649 UTC,"""humidity""",94.17,"""bme280_0""","""mush/controllers/C1/sensors/bm…"
…,…,…,…,…
2025-03-13 23:57:11.243800 UTC,"""temperature""",68.51,"""scd_0""","""mush/controllers/C3/sensors/sc…"
2025-03-13 23:57:49.471732 UTC,"""temperature""",68.46,"""scd_0""","""mush/controllers/C3/sensors/sc…"
2025-03-13 23:58:27.679373 UTC,"""temperature""",68.44,"""scd_0""","""mush/controllers/C3/sensors/sc…"
